## TensorFlow 및 기타 라이브러리 가져오기

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [2]:
import pathlib
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)

In [3]:
batch_size = 32
img_height = 180
img_width = 180

In [4]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 3670 files belonging to 5 classes.
Using 2936 files for training.


In [5]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 3670 files belonging to 5 classes.
Using 734 files for validation.


In [6]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [7]:
num_classes = 5

# ResNet50V2, Xception MobileNetV2 InceptionV3 DenseNet121 중 1개를 사용해 보세요
base_model = tf.keras.applications.ResNet50V2(input_shape=(img_height, img_width, 3),
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False

model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255),
  base_model,
  tf.keras.layers.GlobalAveragePooling2D(),
  layers.Dense(5)
])

In [8]:
base_learning_rate = 0.0002
model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/10
92/92 [==============================] - 24s 204ms/step - loss: 1.4088 - accuracy: 0.4714 - val_loss: 0.8851 - val_accuracy: 0.6689
Epoch 2/10
92/92 [==============================] - 16s 176ms/step - loss: 0.7460 - accuracy: 0.7350 - val_loss: 0.6652 - val_accuracy: 0.7793
Epoch 3/10
92/92 [==============================] - 16s 176ms/step - loss: 0.5893 - accuracy: 0.7864 - val_loss: 0.5783 - val_accuracy: 0.8038
Epoch 4/10
92/92 [==============================] - 16s 176ms/step - loss: 0.5078 - accuracy: 0.8205 - val_loss: 0.5280 - val_accuracy: 0.8256
Epoch 5/10
92/92 [==============================] - 16s 176ms/step - loss: 0.4515 - accuracy: 0.8406 - val_loss: 0.4949 - val_accuracy: 0.8338
Epoch 6/10
92/92 [==============================] - ETA: 0s - loss: 0.4093 - accuracy: 0.8590

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()